<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [6]:
#Установка нужных пакетов
!pip install --upgrade nltk gensim bokeh umap-learn

import itertools
import string

import numpy as np
import umap

from nltk.tokenize import WordPunctTokenizer
from matplotlib import pyplot as plt

from IPython.display import clear_output

import nltk
nltk.download('wordnet')
nltk.download('omw-1.4')
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer
from nltk.tokenize import word_tokenize

ps = PorterStemmer()
lemmatizer = WordNetLemmatizer()

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/vasilijpancenko/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     /Users/vasilijpancenko/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [8]:
# выгружаем датасет:
# !wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1  -O ./quora.txt -nc
data = list(open("./quora.txt", encoding="utf-8"))
data = data[:1000]

In [9]:
tokenizer = WordPunctTokenizer()
print(tokenizer.tokenize(data[50]))

['What', 'TV', 'shows', 'or', 'books', 'help', 'you', 'read', 'people', "'", 's', 'body', 'language', '?']


#Задание 1: Перевести все слова в нижний регистр (NLTK) из data и добавьте как лист токенов в листе data_tok


In [ ]:
# NLP2_1 https://www.hackerrank.com/challenges/detect-the-email-addresses/problem?isFullScreen=true

# import re

# N = int(input())
# result = []

# for _ in range(N):
#     result.extend(re.findall(r'[\w.]+@[\w.]+\.\w{1,3}',input()))
# print(';'.join(sorted(set(result))))

In [10]:
data_tok = [[word.lower() for word in tokenizer.tokenize(line)] for line in data]

# lemmatizer.lemmatize('cats')


for item in data:
    data_item = []
    for token in item:
        data_item.append(str(item).lower())
    data_tok.append(data_item)


#checking

assert all(isinstance(row, (list, tuple)) for row in data_tok), "please convert each line into a list of tokens (strings)"
is_latin = lambda tok: all('a' <= x.lower() <= 'z' for x in tok)
assert all(map(lambda l: not is_latin(l) or l.islower(), map(' '.join, data_tok))), "please make sure to lowercase the data"

Bag Of Words (BOW)

In [11]:
def vectorize(tokens):
    ''' This function takes list of words in a sentence as input 
    and returns a vector of size of filtered_vocab.It puts 0 if the 
    word is not present in tokens and count of token if present.'''
    vector=[]
    for w in filtered_vocab:
        vector.append(tokens.count(w))
    return vector
def unique(sequence):
    '''This functions returns a list in which the order remains 
    same and no item repeats.Using the set() function does not 
    preserve the original ordering,so i didnt use that instead'''
    seen = set()
    return [x for x in sequence if not (x in seen or seen.add(x))]

#create a list of stopwords.You can import stopwords from nltk too
stopwords=["to","is","a"]

#list of special characters.You can use regular expressions too
special_char=[",",":"," ",";",".","?"]

#Write the sentences in the corpus,in our case, just two 
string1="Welcome to Great Learning , Now start learning"
string2="Learning is a good practice"

#convert them to lower case
string1=string1.lower()
string2=string2.lower()

#split the sentences into tokens
tokens1=string1.split()
tokens2=string2.split()
print(tokens1)
print(tokens2)

#create a vocabulary list
vocab=unique(tokens1+tokens2)
print(vocab)

#filter the vocabulary list
filtered_vocab=[]
for w in vocab: 
    if w not in stopwords and w not in special_char: 
        filtered_vocab.append(w)
print(filtered_vocab)

#convert sentences into vectords
vector1=vectorize(tokens1)
print(vector1)
vector2=vectorize(tokens2)
print(vector2)

['welcome', 'to', 'great', 'learning', ',', 'now', 'start', 'learning']
['learning', 'is', 'a', 'good', 'practice']
['welcome', 'to', 'great', 'learning', ',', 'now', 'start', 'is', 'a', 'good', 'practice']
['welcome', 'great', 'learning', 'now', 'start', 'good', 'practice']
[1, 1, 2, 1, 1, 0, 0]
[0, 0, 1, 0, 0, 1, 1]


In [18]:
def filtered(data):
    return [w for w in data if w not in stopwords and w not in special_char]

n = 100
data = data_tok[:n]
vocab = unique([ps.stem(word) for line in data for word in line])
print('vocab:', len(vocab))
filtered_vocab = filtered(vocab)
print('filtered:', len(filtered_vocab))
vectors = [vectorize(line) for line in data]

vocab: 513
filtered: 506


In [19]:
from numpy import dot
from numpy.linalg import norm


a = vectors[0]
b = vectors[10]
cos_sim = dot(a, b)/(norm(a)*norm(b))
cos_sim

0.13453455879926252

Задание 1: Реализовать Bag of words на data_tok (можно на NLTK, можно без)
Сформировать BoW, предварительно сделав стемминг

1.1: Найти самые схожие объекты (косинусное расстояние). Если работаете в colab, то возьмите только первые 100 строчек


Код поиска косинусного расстояние:



```
from numpy import dot
from numpy.linalg import norm

cos_sim = dot(a, b)/(norm(a)*norm(b))

```




In [20]:
cos_sim = 0
for i in range(n):
    for j in range(i, n):
        if i != j:
            a = vectors[i]
            b = vectors[j]
            if cos_sim < dot(a, b)/(norm(a)*norm(b)):
                cos_sim = dot(a, b)/(norm(a)*norm(b))
                a_max = (data[i], i)
                b_max = (data[j], j)

a_max, b_max, cos_sim

((['what', 'are', 'the', 'upcoming', 'mobiles', '?'], 39),
 (['what', 'are', 'the', 'alternatives', 'to', 'walkme', '?'], 70),
 1.0000000000000002)

In [25]:
from numpy import dot
from numpy.linalg import norm

result = []
for i in range(len(vectors)):
    for j in range(len(vectors)):
        if vectors[i] != vectors[j]:
            cos_sim = dot(vectors[i],vectors[j])/(norm(vectors[i])*norm(vectors[j]))
            result+=[[cos_sim,data[i],data[j]]]

sorted(result,key=lambda x:x[0], reverse=True)[0]

[0.8944271909999159,
 ['what', 'are', 'the', 'application', 'of', 'binary', 'search', 'trees', '?'],
 ['what', 'are', 'the', 'components', 'of', 'fitness', '?']]

Задание 2 (дополнительно) : 
Реализовать на базе BoW классификатор спама  

Датасет: https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset/code?datasetId=483&searchQuery=BOW